In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
# tag: remove-cell applied

# Modeling Tumor Data

Dichotomous tumor data is very similar to standard dichotomous tumor data, however, there are a few slight differences:

* The multistage cancer model uses slightly different parameter settings for model fit than the standard multistage model
* A cancer slope factor is calculated
* In some cases, there may be a need to combinue multiple tumor datasets and calculate a single cancer slope factor

To that end, this guide covers some slightly different appraoches that you can use in `pybmds` for handling tumor data.

## Quickstart

To run a single dichotomous dataset:

In [ ]:
import pybmds
from pybmds.models.dichotomous import MultistageCancer

dataset = pybmds.DichotomousDataset(
    doses=[0, 25, 75, 125, 200],
    ns=[20, 20, 20, 20, 20],
    incidences=[0, 1, 7, 15, 19],
    name="Tumor dataset A",
    dose_units="mg/kg-d"
)

model = MultistageCancer(dataset, settings={"bmr": 0.10})
model.execute(slope_factor=True)

print(f"BMD = {model.results.bmd:f}")
print(f"BMDL = {model.results.bmdl:f}")
print(f"CSF = {model.results.slope_factor:f}")

model.plot()

To execute multiple tumor datasets and calculate a single combined slope factor:

In [ ]:
import pybmds
from pybmds.models.multi_tumor import Multitumor

datasets = [
    pybmds.DichotomousDataset(
        doses=[0, 25, 75, 125, 200],
        ns=[20, 20, 20, 20, 20],
        incidences=[0, 1, 7, 15, 19],
        name="Tumor A",
        dose_units="mg/m³"
    ),
    pybmds.DichotomousDataset(
        doses=[0, 25, 75, 125, 200],
        ns=[20, 20, 20, 20, 20],
        incidences=[0, 0, 1, 7, 11],
        name="Tumor B",
        dose_units="mg/m³"
    )
]

session = Multitumor(datasets, settings={"bmr": 0.2}, name="Example")
session.execute()

session.plot()

We can view individual model results for selected models for each dataset as well:

In [ ]:
# Print overall results
print("Overall")
print(f"BMD = {session.results.bmd:f}")
print(f"BMDL = {session.results.bmdl:f}")
print(f"CSF = {session.results.slope_factor:f}")
print()

# Print individual model results
selected_model_indexes = session.results.selected_model_indexes
for i, dataset_models in enumerate(session.models):
    selected_index = selected_model_indexes[i]
    selected_model = dataset_models[selected_index]
    print(f"{selected_model.dataset.metadata.name}: {selected_model.name()}")
    print(f"BMD = {selected_model.results.bmd:f}")
    print(f"BMDL = {selected_model.results.bmdl:f}")
    print(f"CSF = {selected_model.results.slope_factor:f}")
    print()

## Create a tumor dataset

A tumor dataset is created just like a standard dichotomous dataset.

To create a dichotomous dataset, you need a list of doses, incidences, and the total number of subjects. 

You can also add optional atributes, such as `name`, `dose_units`, `response_units`, etc.

In [ ]:
dataset = pybmds.DichotomousDataset(
    name="Chemical X Tumor A",
    dose_units="ppm",
    doses=[0, 25, 75, 125, 200],
    ns=[20, 20, 20, 20, 20],
    incidences=[0, 1, 7, 15, 19],
)

print(dataset.tbl())
dataset.plot()

## Fitting a model to a dataset

With a single tumor dataset defined above, you can run a single multistage cancer model:

In [ ]:
import pybmds
from pybmds.models.dichotomous import MultistageCancer

model = MultistageCancer(dataset, settings={"bmr": 0.10, "degree": 2})
model.execute(slope_factor=True)
model.plot()

After executing, results are stored in a `results` attribute on the model. You can view all results by calling the text method, or atribute atributes individually:

In [ ]:
print(model.name())
print(f"BMD = {session.results.bmd:f}")
print(f"BMDL = {session.results.bmdl:f}")
print(f"CSF = {session.results.slope_factor:f}")

In [ ]:
print(model.text())

### Customizing model settings

Model settings can be customized for a run, just like standard dichotomous models.

In [ ]:
model = MultistageCancer(
    dataset, 
    settings = {
        "bmr_type": pybmds.DichotomousRiskType.AddedRisk,
        "bmr": 0.15, 
        "degree": 3
})
print(model.settings.tbl())

#### Intitial model parameter settings

Initial parameter settings are slightly different for the `MultistageCancer` model than the standard `Multistage`:

In [ ]:
from pybmds.models.dichotomous import Multistage, MultistageCancer

model = Multistage(dataset)
print("Multistage parameter settings:")
print(model.priors_tbl())

model = MultistageCancer(dataset)
print("Multistage Cancer parameter settings:")
print(model.priors_tbl())

For multistage models, the `b2` parameter setting is reused for all beta parameters greater than or equal to b2.

These can be updated:

In [ ]:
model.settings.priors.update('g',  initial_value=0,  min_value=-10, max_value=10)
model.settings.priors.update('b1', initial_value=10, min_value=0,   max_value=100)
model.settings.priors.update('b2', initial_value=20, min_value=0,   max_value=1000)

print(model.priors_tbl())

## Fitting multiple models to a dataset

The example above runs a single Multitumor model to a single dataset. You may want to run multiple multitumor models of varying degrees for example to a single dataset. To do that, follow the guide below, but just fit one dataset instead of multiple (and ignore . 

This is because the model selection critieria for multiple models is only implemented in the Multitumor model below.

## Fitting one or more datasets

To fit multiple models and and one ore more datasets, you'll use an instance of the Multitumor class:

In [ ]:
import pybmds
from pybmds.models.multi_tumor import Multitumor

datasets = [
    pybmds.DichotomousDataset(
        doses=[0, 25, 75, 125, 200],
        ns=[20, 20, 20, 20, 20],
        incidences=[0, 1, 7, 15, 19],
        name="Tumor A",
        dose_units="mg/m³"
    ),
    pybmds.DichotomousDataset(
        doses=[0, 25, 75, 125, 200],
        ns=[20, 20, 20, 20, 20],
        incidences=[0, 0, 1, 7, 11],
        name="Tumor B",
        dose_units="mg/m³"
    )
]

session = Multitumor(datasets)
session.execute()
session.plot()

You can also generate excel and word exports:

In [ ]:
# save excel report
df = session.to_df()
df.to_excel("output/report.xlsx")

# save to a word report
report = session.to_docx()
report.save("output/report.docx")

### Customizing model settings

Settings for all datasets and models should be configured globally and are applied to all models:

In [ ]:
session = Multitumor(
    datasets, 
    settings = {
        "bmr_type": pybmds.DichotomousRiskType.AddedRisk,
        "bmr": 0.15
    }
)

By default, multiple models are executed for each dataset, where the degree is varied from 1 to the number of doses minus 1 (and a maximum of 8). You can specify to run specific degrees only by selecting the desired degree using the `degrees` parameter. Setting a value of 8 runs multiple models, as described above:

In [ ]:
datasets = [
    pybmds.DichotomousDataset(
        doses=[0, 2, 3, 4, 5, 6, 7, 8, 9],
        ns=[20, 20, 20, 20, 20, 20, 20, 20, 20],
        incidences=[0, 1, 4, 8, 11, 12, 13, 14, 15],
        name="Tumor A (9 groups)",
        dose_units="mg/m³"
    ),
    pybmds.DichotomousDataset(
        doses=[0, 2, 3, 4, 5, 6, 7, 8, 9],
        ns=[20, 20, 20, 20, 20, 20, 20, 20, 20],
        incidences=[0, 1, 7, 15, 19, 19, 19, 19, 19],
        name="Tumor B (9 groups)",
        dose_units="mg/m³"
    ),
    pybmds.DichotomousDataset(
        doses=[0, 2, 3, 4, 5],
        ns=[20, 20, 20, 20, 20],
        incidences=[0, 0, 1, 7, 11],
        name="Tumor C (5 groups)",
        dose_units="mg/m³"
    )
]

# will run models 1-8 for the first dataset, only only degree 1 and 2 for other datasets, respectively
degrees=[0, 1, 2]
session = Multitumor(datasets, degrees=degrees)
session.execute()

session.plot()

This analysis ran the following models for each:

In [ ]:
for dataset_models in session.models:
    print(f"{dataset_models[0].dataset.metadata.name}")
    for model in dataset_models:
        print("\t" + model.name())